# 🚀 **JLSatiro Clipper AI - V23.0 Titanium Edition**

Bem-vindo ao **JLSatiro Clipper AI**, a plataforma definitiva para criação de cortes virais automatizados.

### ⚡ **Recursos Titanium:**
- **Smart Crop V2.5**: Rastreamento facial ultra-preciso com MediaPipe (Fixed).
- **Hyper-Speed Transcription**: Transcrição GPU acelerada com Faster-Whisper.
- **Auto-Viral**: Geração automática de Thumbnails e Hooks.
- **100% Local**: Otimizado para rodar localmente no Colab com zero delay.
- **Titanium Vault**: Cache total no Google Drive. Instalação Zero após a primeira execução.

---

In [ ]:
#@title 1. 🛠️ Instalação do Sistema (Titanium Vault)
#@markdown O sistema irá checar seu Drive. Se já existir um backup, ele carregará instantaneamente.

import os
import subprocess
import sys
import shutil
import time

print("💎 [JLSatiro] Iniciando Setup Titanium V23.0...")

# --- DRIVE MOUNT & VAULT CONFIG ---
print("☁️ [0/5] Conectando Cofre (Drive)...\n")
try:
    from google.colab import drive
    drive.mount('/content/drive')
    
    CONFIG_DIR = "/content/drive/MyDrive/JLSatiro_Config"
    VAULT_FILE = os.path.join(CONFIG_DIR, "titanium_libs_v23.tar.gz")
    LIB_PATH = "/content/titanium_libs"
    
    os.makedirs(CONFIG_DIR, exist_ok=True)
    print(f"   ✅ Drive Conectado! Vault em: {VAULT_FILE}")
except Exception as e:
    print(f"   ⚠️ Aviso: Falha ao conectar Drive ({e}). O Modo Persistente falhará.")
    CONFIG_DIR = None
    VAULT_FILE = None
    LIB_PATH = "/usr/local/lib/python3.10/dist-packages" # Fallback default

# Configurar Path Python para as libs customizadas
if LIB_PATH not in sys.path:
    sys.path.insert(0, LIB_PATH)
os.environ["PYTHONPATH"] = f"{LIB_PATH}:{os.environ.get('PYTHONPATH', '')}"

# --- VAULT CHECK ---
vault_restored = False

if VAULT_FILE and os.path.exists(VAULT_FILE) and not os.path.exists(".setup_done"):
    print(f"\n⚡ [VAULT] Backup encontrado no Drive! Restaurando... (Isso é rápido)")
    try:
        # Copy to local (faster extract)
        subprocess.run(f"cp '{VAULT_FILE}' /content/vault.tar.gz", shell=True, check=True)
        
        # Extract
        os.makedirs(LIB_PATH, exist_ok=True)
        print("   📂 Extraindo Bibliotecas...")
        subprocess.run(f"tar -xzf /content/vault.tar.gz -C {LIB_PATH}", shell=True, check=True)
        
        # Cleanup
        os.remove("/content/vault.tar.gz")
        
        # Install System Deps (Must be re-run on fresh Colab instance)
        print("   📦 Restaurando Motores de Sistema (FFmpeg)...")
        subprocess.run("apt-get update -qq && apt-get install -y ffmpeg imagemagick libsndfile1 sox > /dev/null", shell=True)
        
        # Fix Magick
        if os.path.exists("/etc/ImageMagick-6/policy.xml"):
             subprocess.run("sed -i 's/none/read,write/g' /etc/ImageMagick-6/policy.xml", shell=True)

        vault_restored = True
        with open(".setup_done", "w") as f: f.write("restored")
        print("   ✅ SISTEMA RESTAURADO COM SUCESSO!")
    except Exception as e:
        print(f"   ❌ Falha ao restaurar Backup: {e}. Iniciando instalação limpa...")

# --- FRESH INSTALL (If Vault failed or doesn't exist) ---
if not vault_restored and not os.path.exists(".setup_done"):
    # 1. Verificar GPU
    print("\n🖥️ [1/4] Verificando Hardware...")
    try:
        subprocess.run(["nvidia-smi"], check=True)
        print("   ✅ GPU NVIDIA Detectada! Modo Turbo Ativado.")
    except:
        print("   ⚠️ AVISO: GPU não detectada. O sistema rodará.em modo CPU (lento).")
        print("   👉 Dica: Vá em 'Runtime' > 'Change runtime type' > 'T4 GPU'.")

    # 2. Instalar Dependências do Sistema (APT)
    print("📦 [2/4] Instalando Motores de Sistema (FFmpeg, ImageMagick)...")
    subprocess.run("apt-get update -qq", shell=True)
    subprocess.run("apt-get install -y ffmpeg imagemagick libsndfile1 sox -qq", shell=True)

    # CORREÇÃO: ImageMagick Policy p/ MoviePy
    if os.path.exists("/etc/ImageMagick-6/policy.xml"):
        print("   🔧 Aplicando Patch no ImageMagick...")
        subprocess.run("sed -i 's/none/read,write/g' /etc/ImageMagick-6/policy.xml", shell=True)

    # 3. Instalar Bibliotecas Python (PIP) -> TARGET LIB_PATH
    print(f"🐍 [3/4] Instalando Bibliotecas Python em {LIB_PATH}...")
    os.makedirs(LIB_PATH, exist_ok=True)

    # GPU LLM
    print("   🧠 Instalando Llama.cpp Otimizado (GPU/CUDA)...")
    subprocess.run(f'pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu121 --no-cache-dir --target {LIB_PATH}', shell=True)

    libraries = [
        "torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121",
        "faster-whisper>=1.0.0",
        "openai-whisper",
        "mediapipe>=0.10.14",
        "transformers",
        "instructor>=0.5.0,<1.0.0",
        "pydantic",
        "moviepy",
        "opencv-python-headless",
        "librosa",
        "soundfile",
        "imageio",
        "imageio-ffmpeg",
        "pillow",
        "pydub",
        "gradio>=4.0.0,<5.0.0",
        "flask",
        "flask-cors",
        "pytubefix>=6.0.0",
        "yt-dlp",
        "python-dotenv",
        "numpy<2.0.0",
        "pandas",
        "edge-tts"
    ]

    pip_cmd = f"{sys.executable} -m pip install --upgrade --no-cache-dir --target {LIB_PATH}"
    for lib in libraries:
        subprocess.run(f"{pip_cmd} \"{lib}\"", shell=True, stdout=subprocess.DEVNULL)

    print("   ✅ Dependências Python instaladas.")
    
    # 4. CREATE VAULT (BACKUP TO DRIVE)
    if VAULT_FILE:
        print("💾 [VAULT] Criando Backup Inicial no Drive (Isso demora ~1 min, aguarde)...")
        try:
             # Create tar.gz from LIB_PATH
             subprocess.run(f"cd {LIB_PATH} && tar -czf /content/vault.tar.gz .", shell=True, check=True)
             # Copy to Drive
             print("   ☁️ Uploading (~4GB)...")
             shutil.copy("/content/vault.tar.gz", VAULT_FILE)
             print("   ✅ BACKUP SALVO NO DRIVE!")
        except Exception as e:
             print(f"   ⚠️ Erro ao criar backup: {e}")

    with open(".setup_done", "w") as f: f.write("ok")
elif os.path.exists(".setup_done"):
    print("\n⚡ [CACHE] Sistema já ativo e configurado!")


# 5. CONFIG WORKSPACE (Always Run)
print("📂 [5/5] Sincronizando Código & Credenciais...")
repo_url = "https://github.com/noviso123/JLSatiroClipperAI.git"
repo_dir = "/content/JLSatiroClipperAI"

if not os.path.exists(repo_dir):
    subprocess.run(f"git clone {repo_url} {repo_dir}", shell=True)
else:
    subprocess.run(f"cd {repo_dir} && git pull", shell=True)

# RESTORE CREDENTIALS
if CONFIG_DIR:
    files_to_sync = ["client_secret.json", "token.pickle", "cookies.txt"]
    for f_name in files_to_sync:
        src = os.path.join(CONFIG_DIR, f_name)
        dst = os.path.join(repo_dir, f_name)
        if os.path.exists(src):
            if not os.path.exists(dst) or (os.path.getmtime(src) > os.path.getmtime(dst)):
                 shutil.copy(src, dst)
                 print(f"   ♻️ Credencial Restaurada: {f_name}")

# Criar pastas necessárias
os.makedirs(f"{repo_dir}/downloads", exist_ok=True)
os.makedirs(f"{repo_dir}/temp_work", exist_ok=True)
os.makedirs(f"{repo_dir}/backend/models", exist_ok=True)

print("\n✨ SETUP COMPLETO! Execute o próximo bloco.")

In [ ]:
#@title 2. 🚀 Iniciar JLSatiro Clipper AI
#@markdown Clique no play e aguarde. O app irá usar as bibliotecas carregadas do Vault.

import os
import sys
import shutil

# Configurar paths novamente para garantir
LIB_PATH = "/content/titanium_libs"
if os.path.exists(LIB_PATH) and LIB_PATH not in sys.path:
    sys.path.insert(0, LIB_PATH)

# Garantir que estamos no diretório correto
os.chdir("/content/JLSatiroClipperAI")

# Forçar patch do MediaPipe caso o repo não tenha atualizado ainda (Backup Safety)
video_engine_path = "backend/video_engine.py"
if os.path.exists(video_engine_path):
    with open(video_engine_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    if "from mediapipe import solutions" in content:
        print("🔧 Aplicando correção hotfix no MediaPipe (Backup)...")
        content = content.replace(
            "from mediapipe import solutions\n\n        mp_face_detection = solutions.face_detection",
            "import mediapipe as mp\n\n        mp_face_detection = mp.solutions.face_detection"
        )
        content = content.replace(
            "from mediapipe import solutions\n\n        mp_face = solutions.face_detection",
            "import mediapipe as mp\n\n        mp_face = mp.solutions.face_detection"
        )
        with open(video_engine_path, 'w', encoding='utf-8') as f:
            f.write(content)

try:
    # Executar App (Isso bloqueia até o user parar)
    !python app.py
except KeyboardInterrupt:
    pass
finally:
    # BACKUP PARA O DRIVE AO FECHAR
    CONFIG_DIR = "/content/drive/MyDrive/JLSatiro_Config"
    if os.path.exists(CONFIG_DIR):
        print("\n💾 [AUTO-BACKUP] Sincronizando estado final...")
        files_to_sync = ["client_secret.json", "token.pickle", "cookies.txt"]
        for f_name in files_to_sync:
            if os.path.exists(f_name):
                 # Check modification time
                 dst = os.path.join(CONFIG_DIR, f_name)
                 if not os.path.exists(dst) or (os.path.getmtime(f_name) > os.path.getmtime(dst)):
                    shutil.copy(f_name, dst)
                    print(f"   ✅ Salvo: {f_name}")
        print("🏁 Finalizado!")